In [ ]:
import pandas as pd
import chardet
from pathlib import Path

folder_path = Path(r"C:\Users\E1460340\Desktop\New")

# Function to detect encoding
def detect_encoding(file_path):
    with open(file_path, "rb") as f:
        result = chardet.detect(f.read())
    return result['encoding']

# Read datasets
dataframes = {}

for file in folder_path.iterdir():
    if "Dataset" in file.name:
        for dataset in file.iterdir():
            if "Actual" in dataset.name:
                for actual_file in dataset.iterdir():
                    if "Firm and Forecast Orders" in actual_file.name:
                        dataframes['actual_orders'] = pd.read_csv(actual_file, low_memory=False)
                    elif "Shipment" in actual_file.name:
                        dataframes['actual_shipments'] = pd.read_csv(actual_file, low_memory=False)
            
            elif "Prior" in dataset.name:
                for prior_file in dataset.iterdir():
                    if "Firm and Forecast Orders" in prior_file.name:
                        dataframes['prior_orders'] = pd.read_csv(prior_file, low_memory=False)
                    elif "Shipment" in prior_file.name:
                        dataframes['prior_shipments'] = pd.read_csv(prior_file, low_memory=False)

            elif "Demantra" in dataset.name:
                for demantra_file in dataset.iterdir():
                    if "Demantra" in demantra_file.name:
                        encoding = detect_encoding(demantra_file)
                        dataframes['demand_plan'] = pd.read_csv(demantra_file, delimiter="\t", encoding=encoding, low_memory=False)

            elif "factAdjustments" in dataset.name:
                dataframes['adjustments'] = pd.read_excel(dataset, sheet_name="Adjustments")

    elif "Lookup" in file.name:
        for lookup_file in file.iterdir():
            excel_file = pd.ExcelFile(lookup_file)
            tab_list = excel_file.sheet_names
            for tab in tab_list:
                dataframes[tab] = pd.read_excel(lookup_file, sheet_name= tab)

# Rename columns
rename_dict = {
    'SCHEDULE_SHIP_DATE': 'SHIP_DATE',
    'ORDERED_QUANTITY': 'SHIPPED_QUANTITY',
    'Site Cust': 'LEGACY_ACCOUNT_NUMBER',
    'Application Code': 'APPLICATION_CODE',
    'Time': 'SHIP_MONTH',
    'Fiscal Week / Series': 'SHIP_WEEK',
    'DP Plan Proj': 'DP_PLAN'
}

for key in ['actual_orders', 'prior_orders']:
    dataframes[key].rename(columns=rename_dict, inplace=True)

# Merge dataframes
res_actual = pd.concat([dataframes['actual_shipments'], dataframes['actual_orders']], ignore_index=True)
res_prior = pd.concat([dataframes['prior_shipments'], dataframes['prior_orders']], ignore_index=True)

# Rename columns for clarity
res_actual.rename(columns={'SHIPPED_QUANTITY': 'ACTUAL'}, inplace=True)
res_prior.rename(columns={'SHIPPED_QUANTITY': 'PRIOR'}, inplace=True)

# Combine datasets
res_data = pd.concat([res_actual, res_prior], ignore_index=True)

# Apply filtering
res_data = res_data[(res_data['CUSTOMER_TYPE'] == 'External') & (res_data['PORBU'] == 'AC_PORBU')]

# Merge lookup tables
res_data = res_data.merge(dataframes['LOOKUP_RES_PLATFORM'], how='inner', on='PLATFORM')
res_data = res_data.merge(dataframes['LOOKUP_RES_APPCODE'], how='left', left_on='APPLICATION_CODE', right_on='APP_CODE').rename(columns={'SEGMENT_y': 'SEGMENT'})

# Fill missing SEGMENT valuesL
res_data['SEGMENT'] = res_data['SEGMENT'].fillna('OTHERS')

# Remove unnecessary columns
res_df = res_data[['SHIP_DATE', 'SEGMENT', 'LEGACY_ACCOUNT_NUMBER', 'PLATFORM', 'ACTUAL', 'PRIOR']].copy()

# Convert SHIP_DATE to datetime and format properly
res_df['SHIP_DATE'] = pd.to_datetime(res_df['SHIP_DATE'], format='mixed').dt.strftime("%m/%d/%Y")
res_df['SHIP_MONTH'] = pd.to_datetime(res_df['SHIP_DATE']).dt.strftime('%m/1/%Y')
res_df['SHIP_WEEK'] = pd.to_datetime(res_df['SHIP_DATE']).dt.to_period('W').dt.start_time.dt.strftime("%m/%d/%Y")

# Group data to reduce row count
res_df = res_df.groupby(['SHIP_MONTH', 'SHIP_WEEK', 'SEGMENT', 'LEGACY_ACCOUNT_NUMBER']).agg(ACTUAL=('ACTUAL', 'sum'), PRIOR=('PRIOR', 'sum')).reset_index()

# Merge adjustments
adjustments = dataframes['adjustments'].copy()
adjustments['SEGMENT'] = 'RESIDENTIAL'
adjustments['SHIP_MONTH'] = pd.to_datetime(adjustments['SHIP_DATE'], format='%d-%b-%y').dt.strftime('%m/%d/%Y')
adjustments[['ACTUAL', 'PRIOR']] = adjustments[['ACTUAL', 'PRIOR']].fillna(0)

res_adj = adjustments.groupby(['SHIP_MONTH', 'SEGMENT', 'LEGACY_ACCOUNT_NUMBER']).agg(adjustments_ACTUAL=('ACTUAL', 'sum'), adjustments_PRIOR=('PRIOR', 'sum')).reset_index()

df_adj = res_df.merge(res_adj, how='left', on=['SHIP_MONTH', 'SEGMENT', 'LEGACY_ACCOUNT_NUMBER']).fillna(0)
df_adj['ACTUAL'] = df_adj['ACTUAL'] + df_adj['adjustments_ACTUAL']
df_adj['PRIOR'] = df_adj['PRIOR'] + df_adj['adjustments_PRIOR']

# Final dataset
data_complete = df_adj[['SHIP_MONTH', 'SHIP_WEEK', 'SEGMENT', 'LEGACY_ACCOUNT_NUMBER', 'ACTUAL', 'PRIOR']]
data_complete.to_csv(folder_path / 'clean_data.csv', index=False)